# Drug Discovery, FAST-AI Tabular Model test


## DATASETS:
(a) Carbonic Anhydrase II (ChEMBL205), a protein lyase,  
(b) Cyclin-dependent kinase 2 (CHEMBL301), a protein kinase,  
(c) ether-a-go-go-related gene potassium channel 1 (HERG) (CHEMBL240), a voltage-gated ion channel,  
(d) Dopamine D4 receptor (CHEMBL219), a monoamine GPCR,  
(e) Coagulation factor X (CHEMBL244), a serine protease,  
(f) Cannabinoid CB1 receptor (CHEMBL218), a lipid-like GPCR and  
(g) Cytochrome P450 19A1 (CHEMBL1978), a cytochrome P450.  
The activity classes were selected based on data availability and as representatives of therapeutically important target classes or as anti-targets.

In [1]:
!nvidia-smi

Tue Oct  5 15:57:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P2    50W / 240W |   1473MiB /  8116MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Import
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
#%%capture
#!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
#!chmod +x Miniconda3-latest-Linux-x86_64.sh
#!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
#!time conda install -q -y -c conda-forge rdkit

In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem

RDKit WARNING: [15:57:30] Enabling RDKit 2019.09.3 jupyter extensions


In [5]:
top_mcc_scores = {
    
    'CHEMBL205': 0.862,
    'CHEMBL301': 0.926,
    'CHEMBL240': 0.884,
    'CHEMBL219': 0.887,
    'CHEMBL244': 0.983,
    'CHEMBL218': 0.941,
    'CHEMBL1978': 0.904}

In [6]:
path = Path('../dataset/13321_2017_226_MOESM1_ESM/')

In [7]:
#df.head()
list(path.iterdir())

[PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL1978_cl_ecfp_1024.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl-data-with-ecfp-activations.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL244_cl_ecfp_512.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL218_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/RdkitDescriptors.py'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl_ecfp_512.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL301_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl_ecfp_1024.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL1978_cl_ecfp_512.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL240_cl.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL301_cl_ecfp_512.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL218_cl_ecfp_1024.csv'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL301_cl_ecfp_1024.cs

In [8]:
dataset='CHEMBL205_cl'

# TabularPandas

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc,roc_auc_score,recall_score,precision_score,f1_score
from  sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid

In [10]:
from fastai.tabular.all import *

In [35]:
def get_df(dataset, oversample = False):
    df = pd.read_csv(path/f'{dataset}-data-with-ecfp-activations.csv')
    X, y = df.drop(["CID", "SMILES", "Activity", 'Image'], axis=1), df["Activity"]
    dep_var = ['Activity']
    cat_vars = X.columns
    cat_names = list(cat_vars)
    x_train, x_val = train_test_split(df.index, test_size=0.2, random_state=666, stratify=df['Activity'])
    df.loc[x_train, 'is_valid']=False
    df.loc[x_val, 'is_valid']=True
    if oversample:
        df = pd.concat([*[df.loc[(df.is_valid==False) & (df.Activity==1)]]*10, 
                            *[df.loc[(df.is_valid==False) & (df.Activity==0)]], 
                            df.loc[df.is_valid==True]], 
                            ignore_index=True)
    return df, dep_var, cat_vars, cat_names

In [36]:
df, dep_var, cat_vars, cat_names = get_df(dataset)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17941 entries, 0 to 17940
Columns: 1541 entries, CID to act_511
dtypes: bool(1), float64(512), int64(1025), object(3)
memory usage: 210.8+ MB


In [38]:
df.head()

,CID,SMILES,ECFP4_1,ECFP4_2,ECFP4_3,ECFP4_4,ECFP4_5,ECFP4_6,ECFP4_7,ECFP4_8,...,act_502,act_503,act_504,act_505,act_506,act_507,act_508,act_509,act_510,act_511
0,CHEMBL188002,S(=O)(=O)(N)c1cc(N/C(/S)=N\c2cc(C(=O)[O-])c(cc2)C=2c3c(OC4=CC(=O)C=CC=24)cc(O)cc3)ccc1,0,0,0,0,0,0,0,0,...,-5.812351,12.728530,-7.898805,-6.179134,-6.257827,-15.549616,13.285661,10.505152,0.485632,-11.573029
1,CHEMBL364127,Clc1ccc(cc1)C(=O)NC1Cc2cc(S(=O)(=O)N)ccc2C1,0,0,0,0,0,0,0,0,...,-7.801254,12.942183,-7.314398,-12.933152,-6.403533,-31.702936,13.820628,23.262939,9.530989,-24.733044
2,CHEMBL1683469,S(=O)(=O)(N)c1ccc(cc1)CNS(=O)(=O)CC12CCC(CC1=O)C2(C)C,0,0,0,0,0,0,0,0,...,-8.171293,12.266936,-3.845922,-13.375577,-5.168748,-35.100037,18.613527,21.361887,10.445678,-21.621212
3,CHEMBL52564,Oc1ccccc1\C=C\C(=O)[O-],0,0,0,0,0,0,0,0,...,-14.237929,14.534154,-1.153313,-20.543133,-0.235949,-6.748789,-4.842742,15.326861,4.321653,-12.885104
4,CHEMBL21427,OB(O)c1ccc(OC)cc1,0,0,0,0,0,0,0,0,...,-9.017034,19.773624,-11.659730,-16.733728,1.383872,-11.880195,-9.137489,14.942449,4.191879,-12.029409


In [39]:
def get_dls(df, dep_var, cat_vars, cat_names):
    splits = (list(df[df.is_valid == False].index), list(df[df.is_valid == True].index))
    procs = [Categorify]
    to = TabularPandas(df, procs, cat_names=cat_names, y_names=dep_var, splits=splits, y_block = CategoryBlock())
    dls = to.dataloaders()
    return dls

In [40]:
dls = get_dls(df, dep_var, cat_vars, cat_names)

In [41]:
dls.show_batch()

# Loss function with weighted classes

In [ ]:
def get_loss_func(df):
    TARGET = 'Activity'
    class_count_df = df.groupby(TARGET).count()
    n_0, n_1 = class_count_df.iloc[0, 0], class_count_df.iloc[1, 0]
    w_0 = (n_0 + n_1) / (2.0 * n_0)
    w_1 = (n_0 + n_1) / (2.0 * n_1)
    print(w_0)
    print(w_1)
    class_weights=torch.FloatTensor([w_0, w_1]).cuda()
    loss_func = CrossEntropyLossFlat(weight=class_weights)
    return loss_func

In [ ]:
loss_func = get_loss_func(df)

# Train tabular DNN model

In [42]:
def train_model(dls, dataset, loss_func=None, epochs=11, wd=None, layers=[200,100]):
    
    print(f'Training model on dataset: {dataset}')
    print(f'Trained epochs: epochs={epochs}')
    
    learn = tabular_learner(dls, 
                            layers=[200,100],
                            wd=wd,
                            metrics=[accuracy, 
                            F1Score(), 
                            Precision(), 
                            Recall(), 
                            RocAucBinary(), 
                            MatthewsCorrCoef()])
    
    
    #print(f'Finding learning rate...')
    #lr_min, lr_steep = learn.lr_find(suggestions=True, show_plot=False)
    #print(f'Training model with learning rate: {lr_min}')
    learn.fit_one_cycle(epochs)
    
    
    return learn

In [43]:
train_model(dls, dataset, epochs=11)

Training model on dataset: CHEMBL205_cl
Trained epochs: epochs=11


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 7.93 GiB total capacity; 5.55 GiB already allocated; 21.94 MiB free; 5.97 GiB reserved in total by PyTorch)

# Test parameters

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
param_grid = {
    'layers': [[200,100], [300,150], [400, 200], [500,250], [600,300], [700, 350], [1000, 500]]
}
param_grid= ParameterGrid(param_grid)

In [ ]:
def train_with_parameters ():
    datasets = ['CHEMBL205_cl', 'CHEMBL301_cl', 'CHEMBL218_cl', 
                'CHEMBL240_cl', 'CHEMBL219_cl', 
                'CHEMBL244_cl', 'CHEMBL1978_cl']
    i = 0
    
    
    df = pd.read_csv(path/f'{datasets[i]}_ecfp_512.csv')
        
    x_train, x_valid = train_test_split(df.index, test_size=0.2, random_state=42, stratify=df['Activity'])
    splits = (list(x_train), list(x_valid))
    procs = [Categorify]
        
    dep_var = ['Activity']
    X = df.drop(["CID", "SMILES", "Activity"], axis=1)
    cat_vars = X.columns
    cat_names = list(cat_vars)
    
    to = TabularPandas(df, 
                        procs,
                        cat_names=cat_names, 
                        y_names=dep_var, 
                        splits=splits, 
                        y_block = CategoryBlock())
    dls = to.dataloaders()
    
    for setting in param_grid:
        
        print(f"Training with layers: {setting['layers']}")
        learn = train_model(dls, datasets[i], layers=setting['layers'])
        

In [ ]:
train_with_parameters()